# Wine Data Exploration

Some initial exploratory tests


In [20]:
import time
import duckdb as dd
import polars as pl
import glob
import os

con = dd.connect()

current_dir = os.getcwd()
parquet_dir = os.path.join(current_dir, "src", "*.parquet")
parquet_files = glob.glob(parquet_dir)

parquet_path = ", ".join([f"'{file}'" for file in parquet_files])

con.execute(f"CREATE VIEW wine_data AS SELECT * FROM read_parquet([{parquet_path}])")

Some summary tables

In [21]:
# Get the N for each group of data High Quality, Low Quality
con.sql("""
        WITH high_quality AS (
                SELECT COUNT(*) AS high
                FROM wine_data
                WHERE rating > 3
        ), low_quality AS (
                SELECT COUNT(*) AS low
                FROM wine_data
                WHERE rating <= 3
        )
        SELECT high, low
        FROM high_quality
        CROSS JOIN low_quality
        """).show()


┌───────┬───────┐
│ high  │  low  │
│ int64 │ int64 │
├───────┼───────┤
│  7500 │     0 │
└───────┴───────┘



In [3]:
# number of players
con.sql("""SELECT COUNT(DISTINCT White) AS amt_white, COUNT(DISTINCT Black) AS amt_black
            FROM lichess_data
            WHERE Opening ilike '%%' 
                AND Event NOT ilike '%bullet%'
                AND UTCDateTime > '2020-10-25'
            """).show()


┌───────────┬───────────┐
│ amt_white │ amt_black │
│   int64   │   int64   │
├───────────┼───────────┤
│   1923988 │   1889660 │
└───────────┴───────────┘



In [ ]:
# how did the games end?
con.sql("""
        SELECT Termination, COUNT(*)
        FROM lichess_data
        GROUP BY Termination
        ORDER BY COUNT(*) DESC
        """).show()

In [ ]:
# Types of games

con.sql("""
        SELECT Event, COUNT(*)
        FROM lichess_data
        GROUP BY Event
        ORDER BY COUNT(*) DESC
        LIMIT 6
        """).show()


# Plots!

In [6]:
# create dataset for high quality wine 
query = """
    SELECT *
    FROM wine_data
    WHERE quality > 5 
    """

folder_name = "graphs"

file_path = os.path.join(folder_name, "high_quality.csv")

con.execute(f"COPY ({query}) TO '{file_path}' (HEADER, DELIMITER ',')")


In [7]:
# create dataset for low quality wine 
query = """
    SELECT *
    FROM wine_data
    WHERE quality <= 5 
    """

folder_name = "graphs"

file_path = os.path.join(folder_name, "low_quality.csv")

con.execute(f"COPY ({query}) TO '{file_path}' (HEADER, DELIMITER ',')")
